In [10]:
import tensorflow as tf
import numpy as np

In [13]:
from model import birnn
from seq_seq_annot_DS1DS2 import read_mitbih, numerize_y

In [14]:
max_time = 10
classes = ['N', 'S','V']

batch_size = 32
num_units = 128

bidirectional = True

checkpoint_dir = 'checkpoints-seq2seq_DS1DS2'
ckpt_name = 'seq2seq_mitbih_DS1DS2.ckpt'
filename = 'data/s2s_mitbih_aami_DS1DS2'

####################################################################
# deal with data

# read data
print("reading data ...")
X_train, y_train = read_mitbih(filename, max_time, classes=classes, max_nlabel=50000, trainset=1)
X_test, y_test = read_mitbih(filename, max_time, classes=classes, max_nlabel=50000, trainset=0)

# input_dim
input_depth = X_train.shape[2]
n_channels = 10
print("# of sequences: ", len(X_train))

classes = np.unique(y_train)
n_classes = len(classes)

char2numY = dict(zip(classes, range(len(classes))))
char2numY['<GO>'] = len(char2numY)
num2charY = dict(zip(char2numY.values(), char2numY.keys()))
# char2numY = {'N': 0, 'S': 1, 'V': 2, '<GO>': 3}
# num2charY = {0: 'N', 1: 'S', 2: 'V', 3: '<GO>'}

y_train, y_test = numerize_y(y_train, y_test, char2numY)

x_seq_length = len(X_train[0])
y_seq_length = len(y_train[0]) - 1  # why minus one ?

###########################################################
# build the model and train
print("building the model ...")

# Placeholders
inputs = tf.placeholder(tf.float32, [None, max_time, input_depth], name='inputs')
targets = tf.placeholder(tf.int32, (None, None), 'targets')

# decision inputs
dec_inputs = tf.placeholder(tf.int32, (None, None), 'output')

logits = birnn(inputs,
               dec_inputs,
               n_classes + 1,
               n_channels=n_channels,
               input_depth=input_depth,
               num_units=num_units,
               max_time=max_time,
               bidirectional=bidirectional)
# decoder_prediction = tf.argmax(logits, 2)
# confusion = tf.confusion_matrix(labels=tf.argmax(targets, 1), predictions=tf.argmax(logits, 2), num_classes=len(char2numY) - 1)# it is wrong
# mean_accuracy,update_mean_accuracy = tf.metrics.mean_per_class_accuracy(labels=targets, predictions=decoder_prediction, num_classes=len(char2numY) - 1)

with tf.name_scope("optimization"):
    # Loss function
    vars = tf.trainable_variables()
    beta = 0.001
    lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in vars
                       if 'bias' not in v.name]) * beta
    loss = tf.contrib.seq2seq.sequence_loss(logits, targets, tf.ones([batch_size, y_seq_length]))
    # Optimizer
    loss = tf.reduce_mean(loss + lossL2)
    optimizer = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

reading data ...
Records processed!
Records processed!
# of sequences:  5051
building the model ...
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.MaxPooling1D instead.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Pleas

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [15]:
checkpoint_dir = 'checkpoints-seq2seq_DS1DS2'
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    saver = tf.train.Saver()
    saver.restore(sess, tf.train.latest_checkpoint(checkpoint_dir))
    

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from checkpoints-seq2seq_DS1DS2/seq2seq_mitbih_DS1DS2.ckpt


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

2 root error(s) found.
  (0) Not found: Key encoding/bidirectional_rnn/bw/lstm_cell/bias not found in checkpoint
	 [[node save/RestoreV2 (defined at <ipython-input-15-fabf7d7127a8>:5) ]]
  (1) Not found: Key encoding/bidirectional_rnn/bw/lstm_cell/bias not found in checkpoint
	 [[node save/RestoreV2 (defined at <ipython-input-15-fabf7d7127a8>:5) ]]
	 [[save/RestoreV2/_7]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'save/RestoreV2':
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/ipykernel_launcher.py", l